## Pull protein sequences from NCBI
Prior to this, DAVID (DAVID Functional Annotation Bioinformatics Microarray Analysis, https://david.ncifcrf.gov/conversion.jsp ) was used to convert REF_SEQ accession numbers to Protein_GI_Accession numbers

In [ ]:
# import libraries and define associated email for queries
from Bio import Entrez
import time
import pandas as pd

Entrez.email ="karanjia@uw.edu"

In [41]:
# define gis and use Biopython.Entrez to pull protein sequences from NCBI

gis = [15595384, 15595400] # gi = gi number, input is PROTEIN_GI
                         # eventually replace with list
request = Entrez.epost("protein",id=",".join(map(str,gis)))
result = Entrez.read(request)
webEnv = result["WebEnv"]
queryKey = result["QueryKey"]
handle = Entrez.efetch(db="protein",retmode="xml", webenv=webEnv, query_key=queryKey)

d = [] # empty dataframe 

for r in Entrez.parse(handle):
    try:
        gi=int([x for x in r['GBSeq_other-seqids'] if "gi" in x][0].split("|")[1])
    except ValueError:
        gi=None
    #print (">GI ",gi," " + r["GBSeq_primary-accession"] + " " + r["GBSeq_definition"] + "\n" + r["GBSeq_sequence"])
    #print (">GI ",gi," " +  r["GBSeq_sequence"])
    d.append((gi, r['GBSeq_sequence']))
    
pd.DataFrame(d, columns=('GI', 'PROTEIN_SEQ'))

,GI,PROTEIN_SEQ
0,15595384,mkaltssllglfaapvlagllgayvplasaappkeiriavpdvsag...
1,15595400,mhqriasiglgltlalggsaqaagqlnvvswsgyfspqllekfeke...


### Cells below this markdown are rough drafts of the previous set of cells, keep for documentation of different options: 

In [21]:
Entrez.email = "karanjia@uw.edu"


def retrieve_annotation(id_list):

    """Annotates Entrez Gene IDs using Bio.Entrez, in particular epost (to
    submit the data to NCBI) and esummary to retrieve the information.
    Returns a list of dictionaries with the annotations."""

    request = Entrez.epost("gene", id=",".join(id_list))
    try:
        result = Entrez.read(request)
    except RuntimeError as e:
        # FIXME: How generate NAs instead of causing an error with invalid IDs?
        print ("An error occurred while retrieving the annotations.")
        print ("The error returned was %s") % e
        sys.exit(-1)

    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    data = Entrez.esummary(db="gene", webenv=webEnv, query_key=queryKey)
    annotations = Entrez.read(data)

    print ("Retrieved %d annotations for %d genes") % (len(annotations), len(id_list))

    return annotations

In [23]:
retrieve_annotation([879411, 883108])

TypeError: sequence item 0: expected str instance, int found

In [31]:
# don't use this
from Bio import Entrez

def entry_ENTREZ(entrezID): 
    handle = Entrez.efetch(db='nucleotide', id=entrezID, rettype = 'fasta', retmode= 'text')
    f = open('%s.fasta' % singleID, 'w')
    f.write(handle.read())
    handle.close()
    f.close()

#get an id list: this makes a big search and gets a list of id 
handle = Entrez.esearch(db='genome', term="Pseudomonas aeruginosa[Orgn]", idtype="acc")
record = Entrez.read(handle)
handle.close()
print(record["IdList"])

['187']
